# Solar Sunspot Prediction with LSTM

In [9]:
from statsmodels.api import datasets

dta = datasets.sunspots.load_pandas().data
dta.set_index('YEAR', inplace=True)
dta


,SUNACTIVITY
YEAR,
1700.0,5.0
1701.0,11.0
1702.0,16.0
1703.0,23.0
1704.0,36.0
...,...
2004.0,40.4
2005.0,29.8
2006.0,15.2


In [58]:
import numpy as np
WIN_SIZE=10

unrolled = []
for window in dta.rolling(window=WIN_SIZE+1, min_periods=1):
    unrolled.append(window.values.ravel())
    
unrolled = np.array(unrolled[WIN_SIZE:])
print(unrolled)
print('----')
print(unrolled.shape)

[[  5.   11.   16.  ...  10.    8.    3. ]
 [ 11.   16.   23.  ...   8.    3.    0. ]
 [ 16.   23.   36.  ...   3.    0.    0. ]
 ...
 [  8.6  21.5  64.3 ...  40.4  29.8  15.2]
 [ 21.5  64.3  93.3 ...  29.8  15.2   7.5]
 [ 64.3  93.3 119.6 ...  15.2   7.5   2.9]]
----
(299, 11)


In [37]:
X = unrolled[:,:WIN_SIZE, None]
print(X.shape)
y = unrolled[:,WIN_SIZE]
print(y.shape)

(299, 10, 1)
(299,)


In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(WIN_SIZE, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 50)                10400     
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.fit(X, y, epochs=200, verbose=0)

In [60]:
x_input = unrolled[-1,-WIN_SIZE:]
x_input = x_input.reshape((1, WIN_SIZE, 1))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[2.4230595]]


In [49]:
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(WIN_SIZE, 1)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [50]:
model.fit(X, y, epochs=200, verbose=0)

In [51]:
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[89.6129]]
